In [1]:
import pandas as pd
import numpy as np
import os
import warnings
from sklearn import preprocessing

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.utils.class_weight import compute_class_weight
from sklearn import preprocessing

import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

desired_frames = 1
desired_features = 2048

In [2]:
labels = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/data/scene_labels.csv")

In [3]:
labels.head(5)

,SCENE,KEY,SPEAKER,SHOW,Sarcasm,Sarcasm_Type
0,1_10004,1_10004_u,SHELDON,BBT,0.0,NONE
1,1_10009,1_10009_u,PENNY,BBT,0.0,NONE
2,1_1001,1_1001_u,RAJ,BBT,0.0,NONE
3,1_1003,1_1003_u,HOWARD,BBT,1.0,PRO
4,1_10190,1_10190_u,SHELDON,BBT,0.0,NONE


In [4]:
scenes = list(pd.unique(labels["SCENE"]))

In [5]:
len(scenes)

1202

In [6]:
parent_dir = "/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/data/features"
visual_features = {}
context_visual_features = {}
for i in range(len(scenes)):
    # Visual Features
    try:
        vf = np.load(os.path.join(parent_dir, "utterances_final", "resnet_pool5_" + scenes[i] + ".npy"))
        #Global average pooling
        vf_p = np.apply_over_axes(np.mean, vf, [2, 3]) 
        vf_p = np.reshape(vf_p, (vf_p.shape[0],2048))
        if vf_p.shape[0] != 15:
            vf_p = np.pad(vf_p, [(0, 15 - vf_p.shape[0]), (0, 0)], mode='constant')
    except:
        vf_p = np.zeros((15, 2048))
    
    # Visual Features - Context
    try:
        c_vf = np.load(os.path.join(parent_dir, "context_final", "resnet_pool5_" + scenes[i] + "_c.npy"))
        #Global average pooling
        c_vf_p = np.apply_over_axes(np.mean, c_vf, [2, 3]) 
        c_vf_p = np.reshape(c_vf_p, (c_vf_p.shape[0],2048))
        if c_vf_p.shape[0] != 15:
            c_vf_p = np.pad(c_vf_p, [(0, 15 - c_vf_p.shape[0]), (0, 0)], mode='constant')
    except:
        c_vf_p = np.zeros((15, 2048))
    
    visual_features[scenes[i]] = vf_p
    context_visual_features[scenes[i]] = c_vf_p

In [7]:
def get_model_data(context_video_features, video_features):
    model_data = pd.DataFrame(columns=['context_video_feature', 'video_feature','sarcasm','sarcasm_type', 'speaker'])
    for index, row in labels.iterrows():
#         audio_key = row["SCENE"] + "_u.wav"
        model_data = model_data.append({'context_video_feature': context_video_features[row["SCENE"]],
                                    'video_feature': video_features[row["SCENE"]],
                                    'sarcasm' : row["Sarcasm"],
                                    'sarcasm_type' : row["Sarcasm_Type"],
                                    'speaker' : row["SPEAKER"]},
                                  ignore_index=True)
    return model_data

In [8]:
def get_train_test_split(model_data, x_columns, y_column, stratify_column):
    X_train, X_test, Y_train, Y_test = train_test_split(
        model_data[x_columns],
        model_data[y_column],
        train_size=0.8, 
        test_size=0.2, 
        random_state=42, 
        shuffle=True,
        stratify=model_data[stratify_column])
    
    print("Train: ",X_train.shape, Y_train.shape,
      "Test: ",(X_test.shape, Y_test.shape))
    train_data = pd.merge(X_train, Y_train, left_index=True, right_index=True)
    test_data = pd.merge(X_test, Y_test, left_index=True, right_index=True)
    return train_data, test_data

In [9]:
class GRUTensorDataset(Dataset):
    def __init__(self, dataframe, speaker):
        self.data = dataframe
        self.speaker = speaker

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.speaker:
            features = self.data.loc[index, 'padded_video_feature']
            ctxt_features = self.data.loc[index, 'padded_context_video_feature']
            final_features = np.vstack((ctxt_features, features))
            a=np.empty((15, 1))
            a.fill(self.data.loc[index, 'speaker_encode'])
            final_features = np.hstack((features, a))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
        else:
            features = self.data.loc[index, 'padded_video_feature']
            ctxt_features = self.data.loc[index, 'padded_context_video_feature']
            final_features = np.vstack((ctxt_features, features))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(features).float(), label

    def __getindexlist__(self):
        return list(self.data.index.values)
    
class GRUNetSD(nn.Module):
    def __init__(self, input_dim, hidden_dim, 
                 output_dim, n_layers):
        super(GRUNetSD, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, 
                          n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.softmax(self.fc(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, 
                            self.hidden_dim).zero_()
        return hidden
    
class GRUNetSID(nn.Module):
    def __init__(self, input_dim, hidden_dim, 
                 output_dim, n_layers):
        super(GRUNetSID, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, 
                          n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.softmax(self.fc(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, 
                            self.hidden_dim).zero_()
        return hidden
    
def evaluateGRU(gru, review, size):
    hidden = gru.init_hidden(size)
    output, hidden = gru(review, hidden)
    return output

def categoryFromOutput(output):
    top_n, top_i = torch.max(output,dim=1)
    return top_i

def test_accuracy(gru, loader, size):
    actuals = []
    predictions = []
    for data, target in loader:
        output = evaluateGRU(gru, data, size)
        prediction_index = categoryFromOutput(output)
        predictions = prediction_index.tolist()
        actuals = target.tolist()
    return predictions, actuals
    
hidden_size = 15
output_size = 2
input_size_sid = 2048
input_size_sd = 2049
n_layers = 1

In [10]:
warnings.filterwarnings("ignore")

model_data = get_model_data(context_visual_features, visual_features)
# Label Encode Speaker
le = preprocessing.LabelEncoder()
model_data['speaker_encode'] = le.fit_transform(model_data['speaker'])
model_data.head(5)

,context_video_feature,video_feature,sarcasm,sarcasm_type,speaker,speaker_encode
0,"[[0.30626756, 0.66872513, 0.28155187, 0.312841...","[[0.13835047, 0.2704592, 0.44648886, 0.1415337...",0.0,NONE,SHELDON,25
1,"[[0.17222668, 0.7350801, 0.30408216, 0.3297085...","[[0.46479157, 0.1813915, 0.22123067, 0.5245148...",0.0,NONE,PENNY,15
2,"[[0.048892517, 0.23698963, 0.4193368, 0.341263...","[[0.253619, 0.25664786, 0.6646118, 0.4821793, ...",0.0,NONE,RAJ,21
3,"[[0.5546928, 0.36914897, 0.8243424, 0.18547274...","[[0.55624646, 0.16990338, 0.62457716, 0.209021...",1.0,PRO,HOWARD,7
4,"[[0.73847526, 1.0405933, 0.11508263, 0.9313859...","[[0.6140023, 0.4846397, 0.79425097, 0.13518682...",0.0,NONE,SHELDON,25


In [11]:
desired_length = 15
desired_length_ctxt = 15
train_data, test_data = get_train_test_split(model_data, ['context_video_feature', 'video_feature', 'speaker_encode'], 'sarcasm', 'sarcasm')
gru_train = train_data.copy()
gru_test = test_data.copy()
gru_train.reset_index(drop=True, inplace = True)
gru_test.reset_index(drop=True, inplace = True)

        
gru_train['padded_video_feature'] = gru_train.loc[:, 'video_feature']
gru_train['padded_context_video_feature'] = gru_train.loc[:, 'context_video_feature']

gru_test['padded_video_feature'] = gru_test.loc[:, 'video_feature']
gru_test['padded_context_video_feature'] = gru_test.loc[:, 'video_feature']


gru_train["sarcasm"] = gru_train["sarcasm"].astype('int').to_numpy()
gru_test["sarcasm"] = gru_test["sarcasm"].astype('int').to_numpy()

Train:  (961, 3) (961,) Test:  ((241, 3), (241,))


### Speaker Independent

In [12]:
gru_train_tensor = GRUTensorDataset(gru_train[['padded_context_video_feature', 'padded_video_feature', 'sarcasm']], False)
gru_test_tensor = GRUTensorDataset(gru_test[['padded_context_video_feature', 'padded_video_feature', 'sarcasm']], False)

num_of_workers = 0
batch_size = 31
valid_size = 0.1

train_indices = list(range(len(gru_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(gru_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    gru_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices),
    drop_last = True
)

test_loader = torch.utils.data.DataLoader(
    gru_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices),
    drop_last = True
)

test_loader_epoch = torch.utils.data.DataLoader(gru_test_tensor, batch_size=gru_test_tensor.__len__())

In [13]:
gru = GRUNetSID(input_size_sid, hidden_size, output_size, n_layers)
print(gru)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=0.001)

GRUNetSID(
  (gru): GRU(2048, 15, batch_first=True)
  (fc): Linear(in_features=15, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [14]:
n_epochs = 101
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    gru.train()
    for data, target in train_loader:
        h = gru.init_hidden(batch_size)
        optimizer.zero_grad()
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    gru.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        h = gru.init_hidden(batch_size)
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)
    
#     if(epoch%20 == 0):
# #         print(f"Epoch: {epoch+1:02}")
# #         print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
#     if test_loss <= test_min_loss:
# #         print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
# #         torch.save(gru.state_dict(), "fnnmodel.pt")
#         test_min_loss = test_loss
    if(epoch%20 == 0):
        print("Epoch: " + str(epoch))
        predictions, actuals = test_accuracy(gru, test_loader_epoch, gru_test_tensor.__len__())
        print(classification_report(actuals, predictions, digits = 4))

Epoch: 0
              precision    recall  f1-score   support

           0     0.6883    0.4380    0.5354       121
           1     0.5854    0.8000    0.6761       120

    accuracy                         0.6183       241
   macro avg     0.6368    0.6190    0.6057       241
weighted avg     0.6371    0.6183    0.6054       241

Epoch: 20
              precision    recall  f1-score   support

           0     0.6355    0.5620    0.5965       121
           1     0.6045    0.6750    0.6378       120

    accuracy                         0.6183       241
   macro avg     0.6200    0.6185    0.6171       241
weighted avg     0.6201    0.6183    0.6171       241

Epoch: 40
              precision    recall  f1-score   support

           0     0.6355    0.5620    0.5965       121
           1     0.6045    0.6750    0.6378       120

    accuracy                         0.6183       241
   macro avg     0.6200    0.6185    0.6171       241
weighted avg     0.6201    0.6183    0.6171  

### Speaker Dependent

In [15]:
gru_train_tensor = GRUTensorDataset(gru_train[['padded_context_video_feature', 'padded_video_feature', 'speaker_encode','sarcasm']], True)
gru_test_tensor = GRUTensorDataset(gru_test[['padded_context_video_feature', 'padded_video_feature', 'speaker_encode','sarcasm']], True)

num_of_workers = 0
batch_size = 31
valid_size = 0.1

train_indices = list(range(len(gru_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(gru_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    gru_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices),
    drop_last=True
)

test_loader = torch.utils.data.DataLoader(
    gru_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices),
    drop_last=True
)

test_loader_epoch = torch.utils.data.DataLoader(gru_test_tensor, batch_size=gru_test_tensor.__len__())

In [16]:
gru = GRUNetSD(input_size_sd, hidden_size, output_size, n_layers)
print(gru)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=0.001)

GRUNetSD(
  (gru): GRU(2049, 15, batch_first=True)
  (fc): Linear(in_features=15, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [17]:
n_epochs = 101
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    gru.train()
    for data, target in train_loader:
        h = gru.init_hidden(batch_size)
        optimizer.zero_grad()
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    gru.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        h = gru.init_hidden(batch_size)
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)
    
#     if(epoch%20 == 0):
# #         print(f"Epoch: {epoch+1:02}")
# #         print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
#     if test_loss <= test_min_loss:
# #         print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
# #         torch.save(gru.state_dict(), "fnnmodel.pt")
#         test_min_loss = test_loss
    if(epoch%20 == 0):
        print("Epoch: " + str(epoch))
        predictions, actuals = test_accuracy(gru, test_loader_epoch, gru_test_tensor.__len__())
        print(classification_report(actuals, predictions, digits = 4))

Epoch: 0
              precision    recall  f1-score   support

           0     0.5625    0.6694    0.6113       121
           1     0.5876    0.4750    0.5253       120

    accuracy                         0.5726       241
   macro avg     0.5751    0.5722    0.5683       241
weighted avg     0.5750    0.5726    0.5685       241

Epoch: 20
              precision    recall  f1-score   support

           0     0.6471    0.4545    0.5340       121
           1     0.5769    0.7500    0.6522       120

    accuracy                         0.6017       241
   macro avg     0.6120    0.6023    0.5931       241
weighted avg     0.6121    0.6017    0.5928       241

Epoch: 40
              precision    recall  f1-score   support

           0     0.5913    0.5620    0.5763       121
           1     0.5794    0.6083    0.5935       120

    accuracy                         0.5851       241
   macro avg     0.5853    0.5852    0.5849       241
weighted avg     0.5854    0.5851    0.5848  